In [66]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

client = Elasticsearch('http://elasticsearch:9200')

PATH = "restaurants.csv"
INDEX_NAME = "icomida"
MAX_ANALYZED_OFFSET = 2000000

def generator(df, index_name, limit = None):

    if limit is not None:
        df = df.head(limit)

    replace_values = {
        "Bras�_lia": "Brasília",
        "S��o": "São",
        "Indian Rupees(Rs.)": "Rs.",
        "Emirati Diram(AED)": "AED",
        "Dollar($)": "$",
        "Brazilian Real(R$)": "R$",
        "Botswana Pula(P)": "P"
    }

    df = df.replace(replace_values, regex = True)

    df['Coordinates'] = df.apply(lambda row: f"{row.Latitude}, {row.Longitude}", axis = 1)

    df = df.drop(['CountryCode', 'Address', 'Locality', "LocalityVerbose", 'Longitude', 'Latitude', 'HasTableBooking', 'HasOnlineDelivery', 'IsDeliveringNow', 'SwitchToOrderMenu', 'RatingText', "RatingColor"], axis = 1)

    df = df.to_dict(orient='records')
    
    for c, line in enumerate(df):
        yield {
            '_index': index_name,
            '_id': line.get("RestaurantID", None),
            '_source': {
                'restaurant_name': line.get('RestaurantName', ''),
                'city': line.get('City', ''),
                'coordinates': line.get('Coordinates', ''),
                'cuisines': line.get('Cuisines', ''),
                'average_cost_for_two': line.get('AverageCostForTwo', ''),
                'currency': line.get('Currency', ''),
                'price_range': line.get('PriceRange', ''),
                'aggregate_rating': line.get('AggregateRating', ''),
                'votes': line.get('Votes')
            }
        }

In [59]:
replace_values = {
        "Bras�_lia": "Brasília",
        "S��o": "São",
        "Indian Rupees(Rs.)": "Rs.",
        "Emirati Diram(AED)": "AED",
        "Dollar($)": "$",
        "Brazilian Real(R$)": "R$",
        "Botswana Pula(P)": "P"
    }

df = pd.read_csv(PATH)

df = df.replace(replace_values, regex = True)

df['Coordinates'] = df.apply(lambda row: f"{row.Latitude}, {row.Longitude}", axis = 1)

df = df.drop(['CountryCode', 'Address', 'Locality', "LocalityVerbose", 'Longitude', 'Latitude', 'HasTableBooking', 'HasOnlineDelivery', 'IsDeliveringNow', 'SwitchToOrderMenu', 'RatingText', "RatingColor"], axis = 1)

df = df.to_dict(orient='records')

# data_generator = generator(df, INDEX_NAME)

In [63]:
df = pd.read_csv(PATH)
df = df.dropna()
df.isna().sum()

RestaurantID         0
RestaurantName       0
CountryCode          0
City                 0
Address              0
Locality             0
LocalityVerbose      0
Longitude            0
Latitude             0
Cuisines             0
AverageCostForTwo    0
Currency             0
HasTableBooking      0
HasOnlineDelivery    0
IsDeliveringNow      0
SwitchToOrderMenu    0
PriceRange           0
AggregateRating      0
RatingColor          0
RatingText           0
Votes                0
dtype: int64

In [67]:
data_generator = generator(df, INDEX_NAME)

In [68]:
for i in data_generator:
    print(i)

{'_index': 'icomida', '_id': 6317637, '_source': {'restaurant_name': 'Le Petit Souffle', 'city': 'Makati City', 'coordinates': '14.565443, 121.027535', 'cuisines': 'French, Japanese, Desserts', 'average_cost_for_two': 1100, 'currency': 'Botswana Pula(P)', 'price_range': 3, 'aggregate_rating': 4.8, 'votes': 314}}
{'_index': 'icomida', '_id': 6304287, '_source': {'restaurant_name': 'Izakaya Kikufuji', 'city': 'Makati City', 'coordinates': '14.553708, 121.014101', 'cuisines': 'Japanese', 'average_cost_for_two': 1200, 'currency': 'Botswana Pula(P)', 'price_range': 3, 'aggregate_rating': 4.5, 'votes': 591}}
{'_index': 'icomida', '_id': 6300002, '_source': {'restaurant_name': 'Heat - Edsa Shangri-La', 'city': 'Mandaluyong City', 'coordinates': '14.581404, 121.056831', 'cuisines': 'Seafood, Asian, Filipino, Indian', 'average_cost_for_two': 4000, 'currency': 'Botswana Pula(P)', 'price_range': 4, 'aggregate_rating': 4.4, 'votes': 270}}
{'_index': 'icomida', '_id': 6318506, '_source': {'restaura